# **Understanding Token Counts & Context Window:**

In [1]:
from dotenv import load_dotenv
import os



load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## **Tokens:**

Tokens are the basic units of text that models process. They can be as short as one character or as long as one word. <br>
For example:

* The word `"ChatGPT"` is one token.
* The sentence `"Hello, world!"` is four tokens: `"Hello"`, `","`, `"world"`, and `"!"`.

Understanding tokenization is essential because models have limits on the number of tokens they can handle.

## **Context Window:**

A **context window** is the maximum number of tokens (chunks of text) a language model can consider at once when generating a response. It defines how much information (both input and output) the model can "remember" in a single request.

<br>

**1.** **Context Window (also called `"context length"`):**
* This is the **maximum number of tokens** a language model can handle at once.
* It includes **everything in a single prompt-response cycle**:
    * System prompt
    * Few-shot examples
    * User message
    * Model output
* 📌 Yes, it includes both `input tokens` and `output tokens`.

<br>

**2.** **Input Tokens:**
* These are the tokens you send to the model.
* Includes:
    * System instructions
    * Previous chat history (if any)
    * Current user question
    * Few-shot examples
* 📎 This is what the model "reads" before generating a response.

<br>

**3.** **Output Tokens:**
* These are the tokens the model generates in response.
* The output is counted against the context window too.
* 📎 This is what the model "writes" back to you.

<br>

**🔁 Relation:**
* `Context Window = Input Tokens + Output Tokens`

<br>

**📘 Example (GPT-4o, 128,000-token context window):**
| Component            | Token Count |
|----------------------|-------------|
| System prompt        | 50 tokens   |
| Few-shot examples    | 1,000 tokens|
| User input           | 100 tokens  |
| Model output         | 800 tokens  |
| **Total (Context)**  | **1,950 tokens** |

<br>

This is well under 128,000, so it works. <br>

But if your **input = 127,000** tokens, you can only generate **1,000 output tokens max**, or the model will truncate or throw an error.

## **Calculate Tokens:**

In [19]:
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate
from IPython.display import display, Markdown
import asyncio
import tiktoken



def load_llm(model_name:str="gpt-4o", temperature: float=0.2, max_tokens:int=2000) -> OpenAI:
    llm = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens=max_tokens)
    return llm

llm = load_llm(model_name="gpt-4o", temperature=0.8, max_tokens=2000)

In [17]:
# Calculate Input Token Counts & Output Token Counts:

def calculate_input_token_counts(prompt: str, user_input:str, model:str="gpt-4o") -> int:
    encoder = tiktoken.encoding_for_model(model)
    prompt_tokens = len(encoder.encode(prompt))
    user_input_tokens = len(encoder.encode(user_input))
    total_tokens = prompt_tokens + user_input_tokens
    return total_tokens


def calculate_output_token_counts(response: str, model:str="gpt-4o") -> int:
    encoder = tiktoken.encoding_for_model(model)
    prompt_tokens = len(encoder.encode(response))
    return prompt_tokens

### **Example 01:**

In [2]:
import tiktoken

def count_tokens(text, model="gpt-3.5-turbo"):
    """Count the number of tokens in a text string."""
    encoder = tiktoken.encoding_for_model(model)
    tokens = encoder.encode(text)
    return len(tokens)



# Example usage
prompt = "Explain the concept of prompt engineering to a junior developer." # Try to experiment with this...
token_count = count_tokens(prompt)
print(f"Token count: {token_count}")

Token count: 12


### **Example 02:**

* Here I use only system_prompt.

In [9]:
# Write System Prompt:
system_prompt = (
    "You are an AI Assistant named 'Lily,' specializing in answering the questions related to INDIAN History. "
    "This AI Assistant is designed by Dibyendu Biswas, a Generative AI Engineer. "
    "You provide clear, step-by-step explanations and encourage students to learn. "
    "If a question is unclear, ask for clarification. "
    "If you don't know the answer, say 'I don't know. "
    "Use 5 to 10 sentences maximum and keep the answer concise. "
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}"),
])

chain = prompt | llm

In [21]:
query = "What is the significance of the Indus Valley Civilization in Indian history?"
response = chain.invoke({"input": query})
response = response.content


display(Markdown(response))
print()

print("Input Token Counts: ", calculate_input_token_counts(prompt=system_prompt, user_input=query))
print("Output Token Counts: ", calculate_output_token_counts(response=response))

The Indus Valley Civilization, also known as the Harappan Civilization, is significant in Indian history as one of the world's earliest urban cultures, flourishing around 3300 to 1300 BCE. It marks the beginning of urbanization on the Indian subcontinent, showcasing advanced city planning, with well-organized streets and sophisticated drainage systems. The civilization is noted for its impressive architectural achievements, including large public buildings, granaries, and the Great Bath of Mohenjo-Daro.

The Indus Valley people were skilled in various crafts such as pottery, bead-making, and metallurgy, and they engaged in long-distance trade with neighboring regions. Their use of standard weights and measures indicates a high level of socio-economic organization. The script of the Indus Valley remains undeciphered, adding an element of mystery to their culture. Overall, the civilization sets a foundational precedent for future Indian societies in terms of technology, trade, and urban development.


Input Token Counts:  106
Output Token Counts:  188


### **Example 03:**

* Here I use both **`system_prompt`** & **`few_shot_prompt`**.

In [26]:
# Write System Prompt:
system_prompt = (
    "You are an AI Assistant named 'Lily,' specializing in answering the questions related to INDIAN History. "
    "This AI Assistant is designed by Dibyendu Biswas, a Generative AI Engineer. "
    "You provide clear, step-by-step explanations and encourage students to learn. "
    "If a question is unclear, ask for clarification. "
    "If you don't know the answer, say 'I don't know. "
    "Use 5 to 10 sentences maximum and keep the answer concise. "
)


# Define Few-Shot Prompt Template using Few Examples:
few_shot_examples = [
    {'input': 'Hello', 'output': 'Hi there! How can I help you today?'},
    {'input': 'What is your name?', 'output': 'I’m Lily, your assistant, specializing in answering the questions related to INDIAN History'},
    {'input': 'How are you?', 'output': 'I’m doing great, thanks for asking! How about you?'},
    {'input': 'Tell me a joke', 'output': 'Why don’t skeletons fight each other? They don’t have the guts!'},
    {'input': 'What time is it?', 'output': 'I’m not able to tell the time, but you can check your device!'},
    {'input': 'Goodbye', 'output': 'Goodbye! Have a great day!'},
    {'input': 'Can you help me?', 'output': 'Of course! What do you need help with?'},
    {'input': 'What is the weather like?', 'output': 'I’m unable to check real-time weather, but you can look it up online!'},
    {'input': 'Set a reminder', 'output': 'I can’t set reminders, but I can help you with other things!'},
    {'input': 'Where are you from?', 'output': 'I don’t have a specific place, but I’m here to assist you wherever you are!'},
    {'input': 'What can you do?', 'output': 'I can answer questions, tell jokes, help with tasks, and more! What do you need today?'},
    {'input': 'Help me with math', 'output': 'Sure! What math problem would you like help with?'},
    {'input': 'I am bored', 'output': 'Let’s do something fun! Want to hear a joke or maybe try a quiz?'},
    {'input': 'What is 5 + 7?', 'output': '5 + 7 equals 12!'},
    {'input': 'How do I cook pasta?', 'output': 'Boil water, add pasta, cook for 8-12 minutes, drain, and enjoy!'},
    {'input': 'Tell me a fun fact', 'output': 'Did you know? A day on Venus is longer than a year on Venus!'},
    {'input': 'What is your favorite color?', 'output': 'I don’t have a favorite color, but I think blue is nice!'}
]

# Few Shot Template:
few_shot_template = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)
# Few Shot Prompt:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=few_shot_template,
    examples=few_shot_examples,
)

# Define Final Prompt Using System prompt and Few Shot Prompt
prompt2 = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    few_shot_prompt,
    ("user", "{input}"),
])


chain2 = prompt2 | llm

In [25]:
# Calculate the Input Token Counts & Output Token Counts:

# Combine few-shot examples into one formatted string (mimicking the actual prompt sent to the model)
def format_few_shot_examples(examples:list):
    formatted = ""
    for ex in examples:
        formatted += f"Human: {ex['input']}\nAI: {ex['output']}\n"
    return formatted
# format_few_shot_examples(examples=few_shot_examples)



# Calculate total input tokens
def calculate_input_token_counts(system_prompt:str, user_input:str, examples:list, model:str="gpt-4o") -> int:
    encoder = tiktoken.encoding_for_model(model)

    # Format the full input message
    few_shot_text = format_few_shot_examples(examples=examples)
    full_prompt = f"{system_prompt}\n\n{few_shot_text}\nHuman: {user_input}\n"

    return len(encoder.encode(full_prompt))



# Calculate tokens in the output (response)
def calculate_output_token_counts(response:str, model:str="gpt-4o") -> int:
    encoder = tiktoken.encoding_for_model(model)
    return len(encoder.encode(response))


In [27]:
query = "What is the significance of the Indus Valley Civilization in Indian history?"
response = chain2.invoke({"input": query})
response = response.content


display(Markdown(response))
print()

print("Input Token Counts: ", calculate_input_token_counts(system_prompt=system_prompt, user_input=query, examples=few_shot_examples))
print("Output Token Counts: ", calculate_output_token_counts(response=response))

The Indus Valley Civilization, one of the world's oldest urban cultures, is significant in Indian history for several reasons:

1. **Urban Planning**: It showcased advanced urban planning with well-organized cities like Harappa and Mohenjo-Daro, featuring grid layouts, drainage systems, and brick houses.

2. **Trade and Economy**: The civilization engaged in extensive trade with neighboring regions, indicating a robust economy.

3. **Writing System**: It had a script that remains undeciphered, suggesting a complex communication system.

4. **Architecture and Art**: The civilization is known for its impressive architecture and artifacts, including pottery, seals, and sculptures.

5. **Cultural Influence**: It laid the cultural and technological foundation for subsequent Indian societies. 

If you'd like to know more about any of these points, feel free to ask!


Input Token Counts:  510
Output Token Counts:  172


### **Example 03:**

* Here I use both `system_prompt`, `few_shot_prompt` & `memory` concepts:

In [28]:
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage, AIMessage

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from IPython.display import display, Markdown

In [52]:
# Write System Prompt:
system_prompt = (
    "You are an AI Assistant named 'Lily,' specializing in answering the questions related to INDIAN History. "
    "This AI Assistant is designed by Dibyendu Biswas, a Generative AI Engineer. "
    "You provide clear, step-by-step explanations and encourage students to learn. "
    "If a question is unclear, ask for clarification. "
    "If you don't know the answer, say 'I don't know. "
    "Use 5 to 10 sentences maximum and keep the answer concise. "
)


# Define Few-Shot Prompt Template using Few Examples:
few_shot_examples = [
    {'input': 'Hello', 'output': 'Hi there! How can I help you today?'},
    {'input': 'What is your name?', 'output': 'I’m Lily, your assistant, specializing in answering the questions related to INDIAN History'},
    {'input': 'How are you?', 'output': 'I’m doing great, thanks for asking! How about you?'},
    {'input': 'Tell me a joke', 'output': 'Why don’t skeletons fight each other? They don’t have the guts!'},
    {'input': 'What time is it?', 'output': 'I’m not able to tell the time, but you can check your device!'},
    {'input': 'Goodbye', 'output': 'Goodbye! Have a great day!'},
    {'input': 'Can you help me?', 'output': 'Of course! What do you need help with?'},
    {'input': 'What is the weather like?', 'output': 'I’m unable to check real-time weather, but you can look it up online!'},
    {'input': 'Set a reminder', 'output': 'I can’t set reminders, but I can help you with other things!'},
    {'input': 'Where are you from?', 'output': 'I don’t have a specific place, but I’m here to assist you wherever you are!'},
    {'input': 'What can you do?', 'output': 'I can answer questions, tell jokes, help with tasks, and more! What do you need today?'},
    {'input': 'Help me with math', 'output': 'Sure! What math problem would you like help with?'},
    {'input': 'I am bored', 'output': 'Let’s do something fun! Want to hear a joke or maybe try a quiz?'},
    {'input': 'What is 5 + 7?', 'output': '5 + 7 equals 12!'},
    {'input': 'How do I cook pasta?', 'output': 'Boil water, add pasta, cook for 8-12 minutes, drain, and enjoy!'},
    {'input': 'Tell me a fun fact', 'output': 'Did you know? A day on Venus is longer than a year on Venus!'},
    {'input': 'What is your favorite color?', 'output': 'I don’t have a favorite color, but I think blue is nice!'}
]

# Few Shot Template:
few_shot_template = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}")
    ]
)
# Few Shot Prompt:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=few_shot_template,
    examples=few_shot_examples,
)

# Define Final Prompt Using System prompt and Few Shot Prompt
prompt3 = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    few_shot_prompt,
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])


chain3 = prompt3 | llm

In [53]:
# Store the chat history in a dictionary:

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [54]:
# conversational_rag_chain:

conversational_chain = RunnableWithMessageHistory(
    chain3,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [55]:
# Write a Dunction to Calculate the Input Token Counts & Output Token Counts:

def count_tokens(text, model="gpt-4o"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def get_token_usage_from_store(session_id:str, system_prompt:str=system_prompt, 
                               few_shot_examples:list=few_shot_examples, history:dict=store, model="gpt-4o"):
    
    history = history[session_id].messages
    if not history:
        return 0, 0
    
    system_text = system_prompt
    few_shot_text = "\n".join(f"Human: {ex['input']}\nAI: {ex['output']}" for ex in few_shot_examples)
    chat_history_text = ""

    for msg in history[:-1]:  # exclude last if AI response
        role = "Human" if msg.type == "human" else "AI"
        chat_history_text += f"{role}: {msg.content}\n"

    last_user = [m for m in history if m.type == "human"][-1].content
    last_ai = [m for m in history if m.type == "ai"][-1].content # Last Resoinse that is AI's response


    full_prompt_text = f"System: {system_text}\n{few_shot_text}\n{chat_history_text}Human: {last_user}"


    input_tokens = count_tokens(full_prompt_text, model=model)
    output_tokens = count_tokens(last_ai, model=model)

    print("Input Token Counts: ", input_tokens)
    print("Output Token Counts: ", output_tokens)
    

In [56]:
ans = conversational_chain.invoke(
    {"input": "What is INDIA?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

In [57]:
display(Markdown(ans.content))
print()

get_token_usage_from_store(session_id="abc123")

India is a country located in South Asia. It is the seventh-largest country by land area and the second-most populous country in the world. India has a rich and diverse history, with ancient civilizations like the Indus Valley Civilization and historic empires such as the Maurya and Gupta Empires. It gained independence from British rule in 1947 and is known for its cultural diversity, languages, traditions, and significant contributions to art, science, and philosophy. The capital city is New Delhi, and the country is known for landmarks such as the Taj Mahal and its vibrant festivals.


Input Token Counts:  508
Output Token Counts:  116


In [58]:
ans = conversational_chain.invoke(
    {"input": "Hey, My Name is Shyam, I am working in TCS, as a Software Engineer. I am from Kolkata, West Bengal, India."},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

display(Markdown(ans.content))
print()

get_token_usage_from_store(session_id="abc123")

Hi Shyam! It's great to meet you. Working at TCS as a Software Engineer sounds exciting. Kolkata is a city rich in culture and history. How can I assist you today?


Input Token Counts:  684
Output Token Counts:  38


In [59]:
ans = conversational_chain.invoke(
    {"input": "Tell me about Shyam in your recent conversation."},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)

display(Markdown(ans.content))
print()

get_token_usage_from_store(session_id="abc123")

You mentioned that your name is Shyam, you work as a Software Engineer at TCS, and you're from Kolkata, West Bengal, India. Is there anything else you'd like to add or discuss?


Input Token Counts:  716
Output Token Counts:  40


In [60]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is INDIA?', additional_kwargs={}, response_metadata={}), AIMessage(content='India is a country located in South Asia. It is the seventh-largest country by land area and the second-most populous country in the world. India has a rich and diverse history, with ancient civilizations like the Indus Valley Civilization and historic empires such as the Maurya and Gupta Empires. It gained independence from British rule in 1947 and is known for its cultural diversity, languages, traditions, and significant contributions to art, science, and philosophy. The capital city is New Delhi, and the country is known for landmarks such as the Taj Mahal and its vibrant festivals.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 570, 'total_tokens': 687, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'reject

#### **Extract the Tokens count from ChatHistory:**

In [64]:
# Function to process the chat history
def extract_chat_details(chat_history):
    details = []

    # Loop through the dictionary's values (which are InMemoryChatMessageHistory instances)
    for key, chat_data in chat_history.items():
        # Accessing the 'messages' attribute from the InMemoryChatMessageHistory instance
        if hasattr(chat_data, 'messages'):
            messages = chat_data.messages  # Access the messages list

            # Process pairs of (Human -> AI)
            for i in range(0, len(messages), 2):  # Assuming pairs (Human -> AI)
                user_message = messages[i]
                ai_message = messages[i+1]  # The next message should be AI's response

                # Extracting relevant data
                user_query = user_message.content
                ai_response = ai_message.content
                token_usage = ai_message.response_metadata.get('token_usage', {})
                input_tokens = token_usage.get('prompt_tokens', 0)
                output_tokens = token_usage.get('completion_tokens', 0)
                total_tokens = token_usage.get('total_tokens', 0)

                details.append({
                    'user_query': user_query,
                    'ai_response': ai_response,
                    'input_tokens': input_tokens,
                    'output_tokens': output_tokens,
                    'total_tokens': total_tokens
                })

    return details

# Function to print chat details
def print_chat_details(details):
    for idx, chat in enumerate(details):
        print(f"Interaction {idx + 1}:")
        print(f"  User Query: {chat['user_query']}")
        print(f"  AI Response: {chat['ai_response']}")
        print(f"  Token Count: Input Tokens: {chat['input_tokens']}, Output Tokens: {chat['output_tokens']}, Total Tokens: {chat['total_tokens']}")
        print('-' * 40)

# Assuming 'store' is a dictionary containing InMemoryChatMessageHistory instances
chat_details = extract_chat_details(chat_history=store)

# Print extracted details
print_chat_details(chat_details)



Interaction 1:
  User Query: What is INDIA?
  AI Response: India is a country located in South Asia. It is the seventh-largest country by land area and the second-most populous country in the world. India has a rich and diverse history, with ancient civilizations like the Indus Valley Civilization and historic empires such as the Maurya and Gupta Empires. It gained independence from British rule in 1947 and is known for its cultural diversity, languages, traditions, and significant contributions to art, science, and philosophy. The capital city is New Delhi, and the country is known for landmarks such as the Taj Mahal and its vibrant festivals.
  Token Count: Input Tokens: 570, Output Tokens: 117, Total Tokens: 687
----------------------------------------
Interaction 2:
  User Query: Hey, My Name is Shyam, I am working in TCS, as a Software Engineer. I am from Kolkata, West Bengal, India.
  AI Response: Hi Shyam! It's great to meet you. Working at TCS as a Software Engineer sounds exci

## **References:**

* https://www.ibm.com/think/topics/context-window
* https://github.com/dibyendubiswas1998/Generative-AI